In [2]:
# Here is where we import all the things we need
# This example is based on Abhishek's Machine Learning Demystified
# https://github.com/shekit/machine-learning-demystified/

import keras
from keras.models import load_model
import numpy as np
import random

Using TensorFlow backend.


In [3]:
# helper function to sample an index from a probability array
# This comes directly from 
# https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [5]:
# We have to build the same dictionaries here from the source text
# We could instead save and load these instead?
maxlen = 40
# read the file and convert to lowercase (must match how we did it during training)
text = open('data/hamlet.txt').read().lower()
# Get a list of all the unique characters
chars = sorted(list(set(text)))
# Two dictionaries
# Lookup a character by index
# Lookup an index by character
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [6]:
# Load the model
model = load_model('model.h5')

In [7]:
# OK, let's generate some new text
# Pick a random part of the source text
start_index = random.randint(0, len(text) - maxlen - 1)
# Diversity or temperature: higher is wackier
diversity = 0.5
# Text we will generate
generated = ''
# Text to feed into the LSTM
sentence = text[start_index: start_index + maxlen]
# Generated text will include that
generated += sentence

# Generate 50 new characters
for i in range(50):
    # First vectorize the text we are feeding in
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.
    # Then get a probability map of possible predictions of the next char
    preds = model.predict(x, verbose=0)[0]
    # Pick one
    next_index = sample(preds, diversity)
    # What character is it?
    next_char = indices_char[next_index]
    # Add it
    generated += next_char
    # Now feed in the previous sentence minus the first char plus the next char
    sentence = sentence[1:] + next_char
generated

'd then i precepts gave her,\nthat she should this is hamlet,\nthere not say you think at the'